# Lab 2: IR

## Part 1: Basic Indexing

### Preparations:
* Download the provided dataset **DSS_Fall2017_Assign1.zip** and extract it to a local folder (e.g., C:\DSS_Fall2017_Assign1)

In [1]:
MATERIALS_DIR = r"C:\DSS_Fall2017_Assign2"

* Import necessary libraries: whoosh(for IR) and os,shutil(for working with files)

In [2]:
from whoosh import index, writing
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
import os, os.path
import shutil

* Paths:

In [3]:
DOCUMENTS_DIR = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\lab1-q1-test-collection\documents")
INDEX_DIR = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\lab1-q1-test-collection\index1")
QUER_FILE = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\lab1-q1-test-collection\topics\air.topics")
QRELS_FILE = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\lab1-q1-test-collection\qrels\air.qrels")
OUTPUT_FILE = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\lab1-q1-test-collection\myres")
TREC_EVAL = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\trec_eval\trec_eval.exe")
INDEX_DIR2 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\lab1-q1-test-collection\index2")
OUTPUT_FILE2 = os.path.join(MATERIALS_DIR, r"DSS_Fall2017_Assign2\lab1-q1-test-collection\myres2")

### Building the Index

In [4]:
# first, define a Schema for the index
mySchema = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer()))

In [5]:
# if index exists - remove it
if os.path.isdir(INDEX_DIR):
    shutil.rmtree(INDEX_DIR)

# create the directory for the index
os.makedirs(INDEX_DIR)

# create index
myIndex = index.create_in(INDEX_DIR, mySchema)

### Indexing the documents

In [6]:
# First, lets review the documents in our sample dataset
!dir $DOCUMENTS_DIR

 Volume in drive C has no label.
 Volume Serial Number is 10DF-697A

 Directory of C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\lab1-q1-test-collection\documents

09/29/2017  12:39 PM    <DIR>          .
09/29/2017  12:39 PM    <DIR>          ..
09/29/2017  12:39 PM            19,703 email01
09/29/2017  12:39 PM            19,844 email02
09/29/2017  12:39 PM            19,059 email03
09/29/2017  12:39 PM            19,548 email04
09/29/2017  12:39 PM            16,693 email05
09/29/2017  12:39 PM            20,332 email06
09/29/2017  12:39 PM            19,995 email07
09/29/2017  12:39 PM            18,075 email08
09/29/2017  12:39 PM            18,621 email09
09/29/2017  12:39 PM            19,884 email10
09/29/2017  12:39 PM            18,336 email14
              11 File(s)        210,090 bytes
               2 Dir(s)  66,223,439,872 bytes free


In [7]:
# first we build a list of all the full paths of the files in DOCUMENTS_DIR
filesToIndex = []
for root, dirs, files in os.walk(DOCUMENTS_DIR):
    filePaths = [os.path.join(root, fileName) for fileName in files if not fileName.startswith('.')]
    filesToIndex.extend(filePaths)

In [8]:
# print the first 5 paths to make sure it worked
print("\n".join(filesToIndex[:5]))

C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\lab1-q1-test-collection\documents\email01
C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\lab1-q1-test-collection\documents\email02
C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\lab1-q1-test-collection\documents\email03
C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\lab1-q1-test-collection\documents\email04
C:\DSS_Fall2017_Assign2\DSS_Fall2017_Assign2\lab1-q1-test-collection\documents\email05


In [9]:
# count files to index
print("number of files:", len(filesToIndex))

number of files: 11


In [10]:
# open writer
myWriter = writing.BufferedWriter(myIndex, period=20, limit=1000)

try:
    # write each file to index
    for docNum, filePath in enumerate(filesToIndex):
        with open(filePath, "r") as f:
            fileContent = f.read()
            myWriter.add_document(file_path = filePath,
                                  file_content = fileContent)
            
            if (docNum % 1000 == 0):
                print("already indexed:", docNum+1)
    print("done indexing.")

finally:
    # save the index
    myWriter.close()

already indexed: 1
done indexing.


### Querying

In [11]:
# define a query parser for the field "file_content" in the index
myQueryParser = QueryParser("file_content", schema=myIndex.schema)
mySearcher = myIndex.searcher()

In [12]:
# run a sample query for the phrase "item"
sampleQuery = myQueryParser.parse("item")
sampleQueryResults = mySearcher.search(sampleQuery, limit=None)

# inspect the result:
# for each document print the rank and the score
for (docnum, result) in enumerate(sampleQueryResults):
    score = sampleQueryResults.score(docnum)
    fileName = os.path.basename(result["file_path"])
    print(fileName, docnum, score)

email01 0 2.6746417187049216


### Evaluation using TREC_EVAL
In order to evaluate our results we will use a topic file - a list of topics we use to evaluate our IR system

In [14]:
# print the topic file
!cat $QUER_FILE

01 ducks
02 ig nobel prizes
03 mathematics
04 flowing hair
05 music
06 AIR TV


We will compare our evaluate our results with a set of judged results(qrels file) using TREC_EVAL 

In [15]:
# print the first 10 lines in the qrels file
!head -n 10 $QRELS_FILE

01 0 email01 0
01 0 email02 0
01 0 email03 0
01 0 email04 1
01 0 email05 1
01 0 email06 1
01 0 email07 0
01 0 email08 0
01 0 email09 0
01 0 email10 0


First we build a file with our results accoring to TREC_EVAL format (see assignment PDF for more details)

In [16]:
# Load topic file - a list of topics(search phrases) used for evalutation
topicsFile = open(QUER_FILE,"r")
topics = topicsFile.read().splitlines()

# create an output file to which we'll write our results
outputTRECFile = open(OUTPUT_FILE, "w")

# for each evaluated topic:
# build a query and record the results in the file in TREC_EVAL format
for topic in topics:
    topic_id, topic_phrase = tuple(topic.split(" ", 1))
    topicQuery = myQueryParser.parse(topic_phrase)
    topicResults = mySearcher.search(topicQuery, limit=None)
    for (docnum, result) in enumerate(topicResults):
        score = topicResults.score(docnum)
        outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))

# close the topic and results file
outputTRECFile.close()
topicsFile.close()

Now, we use TREC_EVAL to compare our results with the provided qrels file

In [1]:
!$TREC_EVAL -q -m recip_rank $QRELS_FILE $OUTPUT_FILE

'$TREC_EVAL' is not recognized as an internal or external command,
operable program or batch file.


## Part b. Evaluating different configurations

### Inspecting our index

In [18]:
# Is it empty?
print("Index is empty?", myIndex.is_empty())

# How many files indexed?
print("Number of indexed files:", myIndex.doc_count())

Index is empty? False
Number of indexed files: 11


In [19]:
# define a reader object on the index
myReader = myIndex.reader()

In [20]:
# print first 5 indexed documents
[(docnum, doc_dict) for (docnum, doc_dict) in myReader.iter_docs()][0:5]

[(0,
  {'file_path': 'C:\\DSS_Fall2017_Assign2\\DSS_Fall2017_Assign2\\lab1-q1-test-collection\\documents\\email01'}),
 (1,
  {'file_path': 'C:\\DSS_Fall2017_Assign2\\DSS_Fall2017_Assign2\\lab1-q1-test-collection\\documents\\email02'}),
 (2,
  {'file_path': 'C:\\DSS_Fall2017_Assign2\\DSS_Fall2017_Assign2\\lab1-q1-test-collection\\documents\\email03'}),
 (3,
  {'file_path': 'C:\\DSS_Fall2017_Assign2\\DSS_Fall2017_Assign2\\lab1-q1-test-collection\\documents\\email04'}),
 (4,
  {'file_path': 'C:\\DSS_Fall2017_Assign2\\DSS_Fall2017_Assign2\\lab1-q1-test-collection\\documents\\email05'})]

In [21]:
# list indexed terms for field "file_content"
[term for term in myReader.field_terms("file_content")][1000:1025]

['Care',
 'Carlos',
 'Carmen',
 'Carnivalesque',
 'Carolina',
 'Case',
 'Cat',
 'Catalysis',
 'Catalyst',
 'Catchers',
 'Cater',
 'Caused',
 'Caveat',
 'CbZF1d0021swQuc57kfqHt',
 'Cechetto',
 'Ceder',
 'Celebratory',
 'Center',
 'Cereal',
 'Ceremony',
 'Cerrahi',
 'Certolizumab',
 'Cervical',
 'Chair',
 'Chalfie']

In [22]:
#how many terms do we have?
print(myReader.field_length("file_content"))

29729


In [24]:
# how many documents have the phrases "bit", blob"
#   in the field "file_content"?
print("# docs with 'bit'", myReader.doc_frequency("file_content", "bit"))
print("# docs with 'are'", myReader.doc_frequency("file_content", "are"))
print("# docs with 'get'", myReader.doc_frequency("file_content", "get"))

# docs with 'bit' 1
# docs with 'are' 11
# docs with 'get' 6


### Text Analyzers

In [25]:
# we start with basic tokenizer
tokenizer = RegexTokenizer()
[token.text for token in tokenizer("We are going to do Text Analysis with whoosh.analysis")]

['We',
 'are',
 'going',
 'to',
 'do',
 'Text',
 'Analysis',
 'with',
 'whoosh.analysis']

In [28]:
# we might want use stemming:
stmAnalyzer = RegexTokenizer() | StemFilter()
[token.text for token in stmAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

['We', 'ar', 'go', 'to', 'do', 'Text', 'Analysi', 'with', 'whoosh.analysi']

In [29]:
# We probably want to lower-case it
# so we add LowercaseFilter
stmLwrAnalyzer = RegexTokenizer() | LowercaseFilter() | StemFilter()
[token.text for token in stmLwrAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

['we', 'ar', 'go', 'to', 'do', 'text', 'analysi', 'with', 'whoosh.analysi']

In [30]:
# we probably want to ignore words like "we", "are", "with" when we index files
# so we add StopFilter to filter stop words
stmLwrStpAnalyzer = RegexTokenizer() | LowercaseFilter() | StopFilter() | StemFilter()
[token.text for token in stmLwrStpAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

['go', 'do', 'text', 'analysi', 'whoosh.analysi']

In [31]:
# we also probably want to break phrases like "whoosh.analysis" into "whoosh" and "analysis"
# so we add IntraWordFilter
stmLwrStpIntraAnalyzer = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() | StemFilter()
[token.text for token in stmLwrStpIntraAnalyzer("We are going to do Text Analysis with whoosh.analysis")]

['go', 'do', 'text', 'analysi', 'whoosh', 'analysi']

### Evaluating the new analyzers

In [32]:
mySchema2 = Schema(file_path = ID(stored=True),
                   file_content = TEXT(analyzer = stmLwrStpIntraAnalyzer))

In [33]:
# if index exists - remove it
if os.path.isdir(INDEX_DIR2):
    shutil.rmtree(INDEX_DIR2)

# create the directory for the index
os.makedirs(INDEX_DIR2)

# create index or open it if already exists
myIndex2 = index.create_in(INDEX_DIR2, mySchema2)

In [34]:
# open writer
myWriter2 = writing.BufferedWriter(myIndex2, period=20, limit=1000)

try:
    # write each file to index
    for docNum, filePath in enumerate(filesToIndex):
        with open(filePath, "r") as f:
            fileContent = f.read()
            myWriter2.add_document(file_path = filePath,
                                  file_content = fileContent)
            
            if (docNum % 1000 == 0):
                print("already indexed:", docNum+1)
    print("done indexing.")

finally:
    # save the index
    myWriter2.close()

already indexed: 1
done indexing.


In [35]:
# define a query parser for the field "file_content" in the index
myQueryParser2 = QueryParser("file_content", schema=myIndex2.schema)
mySearcher2 = myIndex2.searcher()

In [36]:
# Load topic file - a list of topics(search phrases) used for evalutation
topicsFile = open(QUER_FILE,"r")
topics = topicsFile.read().splitlines()

# create an output file to which we'll write our results
outputTRECFile2 = open(OUTPUT_FILE2, "w")

# for each evaluated topic:
# build a query and record the results in the file in TREC_EVAL format
for topic in topics:
    topic_id, topic_phrase = tuple(topic.split(" ", 1))
    topicQuery = myQueryParser2.parse(topic_phrase)
    topicResults = mySearcher2.search(topicQuery, limit=None)
    for (docnum, result) in enumerate(topicResults):
        score = topicResults.score(docnum)
        outputTRECFile2.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))

# close the topic and results file
outputTRECFile2.close()
topicsFile.close()

In [37]:
!$TREC_EVAL -q $QRELS_FILE $OUTPUT_FILE2

num_ret               	01	3
num_rel               	01	3
num_rel_ret           	01	3
map                   	01	1.0000
Rprec                 	01	1.0000
bpref                 	01	1.0000
recip_rank            	01	1.0000
iprec_at_recall_0.00  	01	1.0000
iprec_at_recall_0.10  	01	1.0000
iprec_at_recall_0.20  	01	1.0000
iprec_at_recall_0.30  	01	1.0000
iprec_at_recall_0.40  	01	1.0000
iprec_at_recall_0.50  	01	1.0000
iprec_at_recall_0.60  	01	1.0000
iprec_at_recall_0.70  	01	1.0000
iprec_at_recall_0.80  	01	1.0000
iprec_at_recall_0.90  	01	1.0000
iprec_at_recall_1.00  	01	1.0000
P_5                   	01	0.6000
P_10                  	01	0.3000
P_15                  	01	0.2000
P_20                  	01	0.1500
P_30                  	01	0.1000
P_100                 	01	0.0300
P_200                 	01	0.0150
P_500                 	01	0.0060
P_1000                	01	0.0030
num_ret               	02	11
num_rel               	02	8
num_rel_ret           	02	8
map                   	02	0.9207
Rprec 

In [38]:
# let count the same words again
myReader2 = myIndex2.reader()
print("# docs with 'bit'", myReader2.doc_frequency("file_content", "bit"))
print("# docs with 'are'", myReader2.doc_frequency("file_content", "are"))
print("# docs with 'get'", myReader2.doc_frequency("file_content", "get"))

# docs with 'bit' 11
# docs with 'are' 0
# docs with 'get' 7


** Can you explain the differences? **

### Using NLTK's stemmers and lemmatizers

In [39]:
import nltk
from nltk.stem import *

In [40]:
# download required resources
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ron_suprun\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [41]:
# we'll compare two stemmers and a lemmatizer
lrStem = LancasterStemmer()
sbStem = SnowballStemmer("english")
wnLemm = WordNetLemmatizer()

In [42]:
# define a list of words to compare the stemmers on
listWords = ["going", "saying", "minimize", "maximum", 
             "meeting", "files", "tries", "is", "are", "beautiful",
             "summarize", "better", "dogs", "phenomena"]

In [43]:
for word in listWords:
    print("%15s %15s %15s %15s" % (lrStem.stem(word),
                                   sbStem.stem(word),
                                   wnLemm.lemmatize(word),
                                   wnLemm.lemmatize(word, 'v')))

          going              go           going              go
            say             say          saying             say
          minim           minim        minimize        minimize
          maxim         maximum         maximum         maximum
           meet            meet         meeting            meet
            fil            file            file            file
            tri             tri             try             try
             is              is              is              be
             ar             are             are              be
         beauty          beauti       beautiful       beautiful
           summ          summar       summarize       summarize
            bet          better          better          better
            dog             dog             dog             dog
       phenomen       phenomena      phenomenon       phenomena


### How to use NLTK stemmers / lemmatizers in Whoosh

In [44]:
# Dont change this! Use it as-is in your code
# This filter will run for both the index and the query
from whoosh.analysis import Filter
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

In [45]:
# Example1: Whoosh filter for NLTK's LancasterStemmer
myFilter1 = RegexTokenizer() | CustomFilter(LancasterStemmer().stem)
[token.text for token in myFilter1("We are going to do Text Analysis with whoosh.analysis")]

['we', 'ar', 'going', 'to', 'do', 'text', 'analys', 'with', 'whoosh.analysis']

In [46]:
# Example2: Whoosh filter for NLTK's WordNetLemmatizer
myFilter2 = RegexTokenizer() | CustomFilter(WordNetLemmatizer().lemmatize)
[token.text for token in myFilter2("We are going to do Text Analysis with whoosh.analysis")]

['We',
 'are',
 'going',
 'to',
 'do',
 'Text',
 'Analysis',
 'with',
 'whoosh.analysis']

In [47]:
# Example3: Whoosh filter for NLTK's WordNetLemmatizer for verbs
myFilter3 = RegexTokenizer() | CustomFilter(WordNetLemmatizer().lemmatize, 'v')
[token.text for token in myFilter3("We are going to do Text Analysis with whoosh.analysis")]

['We', 'be', 'go', 'to', 'do', 'Text', 'Analysis', 'with', 'whoosh.analysis']

You can now use myFilter1/2/3 as part of your Schema

------------
You can find details of other NLTK Stemmers and Lemmatizers here:

http://www.nltk.org/api/nltk.stem.html